In [19]:
import pandas as pd
import sys
import os

# Add project root to Python path to allow module imports
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.utils.forecasting import forecast_and_evaluate
from src.utils.cluster_zone import cluster_zones_with_kpi

# Forecast and evaluate

# Hourly

In [20]:
df1 = pd.read_parquet("../processed/cleaned_data/cleaned_yellow_tripdata_2021-01.parquet")

df1_forecast_hourly = forecast_and_evaluate(
    df1=df1,
    freq='H',
    test_periods= 168,
    arima_order=(1, 0, 1)
)

In [21]:
df1_forecast_hourly['metrics']

,MAE,MAPE,RMSE
Baseline,176.363095,0.113524,302.546307
ARIMA,1322.146715,3.601659,1475.926936
Linear Regression,1336.850919,4.825480,1460.990350


In [22]:
df1_forecast_hourly['predictions']

,Actual,Baseline,ARIMA,Linear Regression
hour,,,,
2021-01-25 00:00:00,231,286,379.852381,1997.083617
2021-01-25 01:00:00,119,173,471.706925,1997.883379
2021-01-25 02:00:00,67,83,557.475946,1998.683140
2021-01-25 03:00:00,43,54,637.562619,1999.482902
2021-01-25 04:00:00,88,109,712.343410,2000.282663
...,...,...,...,...
2021-01-31 19:00:00,1484,1347,1766.280102,2127.444738
2021-01-31 20:00:00,932,1148,1766.281392,2128.244500
2021-01-31 21:00:00,770,906,1766.282597,2129.044261


# Daily

In [23]:
df1_forecast_daily = forecast_and_evaluate(
    df1=df1,
    freq='D',
    test_periods=7,        # 7 ngày test
    arima_order=(1, 0, 1)
)

In [24]:
df1_forecast_daily['metrics']

,MAE,MAPE,RMSE
Baseline,3186.714286,0.067892,4380.558167
ARIMA,9947.501167,0.221016,10391.329658
Linear Regression,6657.291470,0.173598,8934.592359


In [25]:
df1_forecast_daily['predictions']

,Actual,Baseline,ARIMA,Linear Regression
day,,,,
2021-01-25,46338,36394,34981.335230,47102.605072
2021-01-26,47005,49962,39585.881917,47479.416812
2021-01-27,51595,48598,41329.613834,47856.228551
2021-01-28,55438,52365,41989.961356,48233.040290
2021-01-29,53623,53207,42240.033541,48609.852029
2021-01-30,38445,41290,42334.735339,48986.663768
2021-01-31,30500,30425,42370.598706,49363.475507


# Cluster Zone

In [26]:
# Load data for clustering
df1 = pd.read_parquet("../processed/cleaned_data/cleaned_yellow_tripdata_2021-01.parquet")

In [27]:
# Perform integrated clustering
qa_flags = pd.read_parquet("../processed/flags_for_analysis/flag_yellow_tripdata_2021-01.parquet")
clustered_df = cluster_zones_with_kpi(df1, qa_flags)

In [28]:
clustered_df[0].head()

,zone,time_bin,duration_p50,duration_p95,trips_index_100,cluster,cluster_name
0,Allerton/Pelham Gardens,Early Morning,29.0,44.00,0.692021,1,Low Demand – Smooth Flow
1,Allerton/Pelham Gardens,Morning,24.0,36.80,1.038032,1,Low Demand – Smooth Flow
2,Allerton/Pelham Gardens,Morning Rush,14.0,35.00,5.997518,2,Efficient High Volume
3,Allerton/Pelham Gardens,Midday,16.0,44.15,15.916489,1,Low Demand – Smooth Flow
4,Allerton/Pelham Gardens,Evening Rush,12.0,41.40,3.114096,2,Efficient High Volume
